# Making WFI Mosaics with RomanCal

***

# Kernel Information

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

# Imports
 Libraries used
- *romancal* for running the processing pipeline
- *roman_datamodels* for opening Roman WFI ASDF files
- *s3fs* for streaming files from an S3 bucket

In [ ]:
import roman_datamodels as rdm
from romancal.pipeline import MosaicPipeline
from romancal.associations import asn_from_list
from astropy.visualization import simple_norm
import s3fs
import glob
import json
import matplotlib.pyplot as plt

# Introduction
The purpose of this notebook is to combine multiple Level 2 (L2; calibrated rate image) data products into a Level 3 (L3; distortion-corrected, co-added image) product. Details about the Roman data levels can be found [here](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). The pipeline for combinging images is called the Mosaic Pipeline, and it consists of the following steps:

1. **Flux step:** Transform pixels from instrumental units of data numbers per second (DN/s) in the L2 products into physical surface brightness units of megaJanskys per steradian (MJy/sr).
2. **Skymatch step:** Determine scaling factors necessary to match the flux levels in the overlapping sections of the input images. The images will be scaled by these factors later in the combination process.
3. **Outlier detection step:** Examine the input images to detect and flag outliers in the pixel values for a given sky position.
4. **Resample step:** Use the drizzle algorithm to combine the input data products. Reject outliers and other undesirable pixels based on data quality flags and oversample the pixels if indicated.

**Note:** L3 data products in the Roman Archive will be tessellated on the sky such that individual L3 files ("sky cells") will be small with some overlap between adjacent cells. More information on the tessellation will be available in the near future. In this tutorial, we will not use the tessellation at this time and the tutorial will be updated in the future to show how to make tessellated and non-tessellated products.

**Note:** An additional step, the Source Catalog step, exists at the end of the Mosaic Pipeline to create Level 4 (L4; catalogs and other high-level products) outputs. In this tutorial, we have disabled the SourceCatalog step. While development continues on the Roman pipeline, enabling the SourceCatalog step may result in an error.

***

# Tutorial Data
In this tutorial, we use L2 WFI data files that are the result of RomanCal processing of L1 simulated files created with Roman I-Sim. If you have already worked through the tutorials "Simulating WFI Imaging Data with Roman I-Sim" and "Calibrating WFI Exposures with RomanCal," then you may already have these files saved locally. If not, then these files are also stored in the science platform S3 bucket. For more information on how to access these data, see the Data Discovery and Access tutorial.

The L2 products that we will use for the tutorial represent two WFI detectors (WFI01 and WFI02) with four different pointings that have small dithers between them. As this has not been optimized for oversampling, and only for gap-filling between detectors, we will create a mosaicked L3 product that uses the native pixel scale.

# Making WFI Mosaics

## Generating the Association File

Association table files (often abbreviated "association files" or "ASN files") are JavaScript Object Notation (JSON) formatted files files that list the input products that should be processed together and the output products that should be created. In the Mosaic Pipeline, association files specifically enumerate the input L2 products that are used to create a single L3 product.

The association generator function `asn_from_list()` can be used to create a properly formatted JSON file for use with the Mosaic Pipeline. The code takes as input as list of L2 products and the name of the L3 output product.

If you have not completed the previous tutorial "Calibrating WFI Exposures with RomanCal," then you can download the data products ...

In [ ]:
# Copy the files from the S3 bucket if they are not in local storage
#
# Uncomment the following lines if you do not have the output files 
# from the tutorial "Calibrating WFI Exposures with RomanCal"
#
#

Next we can create the association JSON file using the `asn_from_list()` function:

In [ ]:
# Collect our input files and make the L3 output filename
input_files = glob.glob('r0003*_????_wfi0[12]_cal.asdf')
product_name = 'my_roman_mosaic'

# Create the ASN JSON object
asn = asn_from_list.asn_from_list([(i, 'science') for i in input_files], 
                                  product_name=product_name, 
                                  with_exptype=True, target='')

Now let's take a look at the JSON object we just created. It should contain a listing of each of our input files as well as the output product name: 

In [ ]:
asn

We can also save the ASN file to disk:

In [ ]:
# Write the ASN file to disk
_, serialized = asn.dump(format='json')
with open('my_roman_mosaic_asn.json', 'w') as f:
    f.write(serialized)

## Running the Mosaic Pipeline

Next we run the Mosaic Pipeline using the `MosaicPipeline` class. As with the L1 to L2 Exposure Pipeline, there are many optional arguments to customize the behavior of the pipeline. At this time, we will not delve into these optional parameters. The input files we have simulated are for a simple gap-filling dither pattern rather than sub-pixel dithering to optimize the point spread function (PSF); thus, we use the default native sampling of the WFI detectors. In the future, we may explore adding more information for optimizing the Mosaic Pipeline output.

In [ ]:
result = MosaicPipeline.call('my_roman_mosaic_asn.json', save_results=True, steps={'sourcecatalog': {'skip': True}})

Now let's open the mosaic file we've created and take a look at what's inside:

In [ ]:
mos_file = rdm.open('my_roman_mosaic_i2d.asdf')
mos_file.info()

In [ ]:
norm = simple_norm(mos_file.data, 'asinh', vmin=0.01, vmax=3.5)
fig, ax = plt.subplots(figsize=(20, 15))
ax.imshow(mos_file.data, origin='lower', norm=norm)

# TO DO

Add info about L3 file contents (context, weight).

# Aditional Resources
- [romanisim](https://romanisim.readthedocs.io/en/latest/index.html)
- [romancal](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)

## About this notebook
**Author:** Tyler Desjardins\
**Updated On:** 2024-12-16

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 